In [ ]:
#Import libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt


In [ ]:
#Define Haversine formula
#Inputs are the coordinates of two points, lon1, lat1, lon2, lat2
#Returns the distance between them

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points  on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r 

In [ ]:
#Read data
#This is a Phyphox GPS measurement. Walking short distance outside. 

data = pd.read_csv('gps_08092023.csv')
data.head()


In [ ]:
#Derive velocity using haversine (although it is already in the data in most cases)

lat = data['Latitude (°)'] #Latitude
lon = data['Longitude (°)'] #Longitude

data['dist'] = np.zeros(lat.shape[0]) #Add new column (of zeros) to the data. We'll calculate the distance between consecutive points there
data['time_diff']  = np.zeros(lat.shape[0]) #Add new column (of zeros) to the data. We'll calculate the time between consecutive points there

for i in range(lat.shape[0]-1): #For-loop goes throught the data untill the second last row
  data['dist'][i] = haversine(lon[i], lat[i],lon[i+1], lat[i+1])*1000 #The distance between data points with indices i ja i + 1 (in meters)
  data['time_diff'][i] = data['Time (s)'][i+1] - data['Time (s)'][i]  #The time step between data points with indices i ja i + 1 (in seconds)

data['dist'][0] = 0
data['velocity'] = data['dist']/data['time_diff']

data.head()

In [19]:
import folium 
my_map = folium.Map(location=[np.mean(data['Latitude (°)']), np.mean(data['Longitude (°)'])], zoom_start=16)

#add a markers if you wants
#for each in location_list:  
#   folium.Marker(each).add_to(my_map)

#Draw lines of motion
folium.PolyLine(data[['Latitude (°)','Longitude (°)']], color="blue", weight=2.5, opacity=1).add_to(my_map)

# Save map
my_map.save("Ilpo_map.html")
my_map

In [ ]:
np.mean(data['Latitude (°)'])